In [ ]:
import torch
import os

# Check if ultralytics is installed, install if not
try:
    from ultralytics import YOLO
except ImportError:
    print("Installing ultralytics...")
    !pip install ultralytics
    from ultralytics import YOLO

In [ ]:
import onnx
print(f"ONNX version: {onnx.__version__}")

In [ ]:
# Path to your fine-tuned PyTorch model
model_path = "yolo11n.pt"  # Update this if your model is in a different location

# Check if model exists
if not os.path.exists(model_path):
    raise FileNotFoundError(f"Model file {model_path} not found")

# Load the model
model = YOLO(model_path)

# Export to ONNX format
# The simplify=True parameter will attempt to simplify the model where possible
# opset is the ONNX operator set version (default is usually fine)
# Other options like dynamic, imgsz, etc. can be adjusted as needed
onnx_path = model_path.replace('.pt', '.onnx')
success = model.export(format="onnx", simplify=True, opset=12)

if success:
    print(f"Model successfully converted to ONNX format and saved as: {onnx_path}")
else:
    print(f"Model failed converted to ONNX format")

In [ ]:
# Try importing onnx for verification
try:
    import onnx

    # Load the ONNX model
    onnx_model = onnx.load(onnx_path)

    # Check if the model is well-formed
    onnx.checker.check_model(onnx_model)

    # Print a success message
    print("ONNX model is valid!")

    # Print some basic information about the model
    print(f"Model IR version: {onnx_model.ir_version}")
    print(f"Producer name: {onnx_model.producer_name}")
    print(f"Producer version: {onnx_model.producer_version}")
    print(f"Model version: {onnx_model.model_version}")

    # Print input and output information
    for i, input_tensor in enumerate(onnx_model.graph.input):
        print(f"Input {i}: {input_tensor.name}")

    for i, output_tensor in enumerate(onnx_model.graph.output):
        print(f"Output {i}: {output_tensor.name}")

except ImportError:
    print("ONNX package not installed. To verify the model install it with: pip install onnx")
except Exception as e:
    print(f"Error verifying ONNX model: {e}")

In [ ]:
try:
    import onnxruntime as ort
    import numpy as np
    import cv2
    from PIL import Image

    # Create an ONNX Runtime session
    session = ort.InferenceSession(onnx_path)

    # Load and preprocess an image (adjust as needed for your model)
    # This is a simplified example and may need adjustments based on your model's specific requirements
    test_image_path = "my_fruits/images/4de76412-image_49.jpg"  # Replace with a test image path

    if os.path.exists(test_image_path):
        # Load and preprocess image - this may vary based on your model requirements
        img = cv2.imread(test_image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (640, 640))  # Common YOLO input size
        img = img.transpose(2, 0, 1)  # HWC to CHW
        img = np.ascontiguousarray(img) / 255.0  # Normalize to 0-1
        img = np.expand_dims(img, 0).astype(np.float32)  # Add batch dimension

        # Get model inputs
        input_name = session.get_inputs()[0].name

        # Run inference
        outputs = session.run(None, {input_name: img})

        print("ONNX model inference successful!")
        print(f"Output shape: {outputs[0].shape}")
    else:
        print(f"Test image {test_image_path} not found. Skipping inference test.")

except ImportError:
    print("onnxruntime package not installed. To test inference install it with: pip install onnxruntime")
except Exception as e:
    print(f"Error running inference with ONNX model: {e}")